# Libraries

In [1]:
import os

import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

import mlflow

import xgboost as xgb

# we use hyperopt here, but optuna in the exercise
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
#Functions
def split_and_read_data(path):
    df = pd.read_csv(path)
    df_list = []

    for col in df.columns:
        if col != "Date":
            df[col] = df[col].str.replace(",", "", regex=False)
            df[col] = pd.to_numeric(df[col], errors='coerce')
    df['Date'] = pd.to_datetime(df['Date'])

    total_len = len(df)
    train_end = int(0.7 * total_len)
    val_end = int(0.85 * total_len)
    
    train_df = df.iloc[:train_end].sort_values("Date")
    val_df = df.iloc[train_end:val_end].sort_values("Date")
    test_df = df.iloc[val_end:].sort_values("Date")

    return prepare_gold_data(train_df), prepare_gold_data(val_df), prepare_gold_data(test_df)


def prepare_gold_data(df, currency_col='EUR', new_col_name='gold_diff'):
    """
    Prepares the DataFrame by keeping only the 'Date', 'EUR' value, and daily difference.
    
    Parameters:
        df (pd.DataFrame): Original DataFrame with 'Date' and currency columns.
        currency_col (str): The currency column to use (e.g., 'EUR').
        new_col_name (str): The name of the new column for daily differences.
    
    Returns:
        pd.DataFrame: Cleaned and prepared DataFrame.
    """
    df = df.copy()
    df[new_col_name] = df[currency_col].diff().fillna(0)
    return df[['Date', currency_col, new_col_name]]


def save_if_not_exists(train_df, val_df, test_df, folder='./data-files'):
    #Will only activate the first time around or if the files are deleted
    os.makedirs(folder, exist_ok=True)

    paths = {
        'train': os.path.join(folder, 'train.csv'),
        'val': os.path.join(folder, 'val.csv'),
        'test': os.path.join(folder, 'test.csv'),
    }

    if not all(os.path.exists(path) for path in paths.values()):
        print("Saving files...")
        if not os.path.exists(paths['train']):
            train_df.to_csv(paths['train'], index=False)
        if not os.path.exists(paths['val']):
            val_df.to_csv(paths['val'], index=False)
        if not os.path.exists(paths['test']):
            test_df.to_csv(paths['test'], index=False)
    else:
        print("Files already exist. Skipping save.")


def make_date_string(train_df, val_df):
    train_df['Date'] = train_df['Date'].astype(str)
    val_df['Date'] = val_df['Date'].astype(str)

    return train_df, val_df


def prepare_regression_train_or_val_data(df, dv, train):
    categorical = ['Date']
    numerical = ['EUR']

    data_dict = df[categorical + numerical].to_dict(orient='records')
    Y_data = df['gold_diff'].values

    if train:
        X_data = dv.fit_transform(data_dict)
    else:
        X_data = dv.transform(data_dict)

    return data_dict, X_data, Y_data


## Preprocessing

In [ ]:
path = './data-files/Daily.csv'

train_df, val_df, test_df = split_and_read_data(path)

print("Train size:", len(train_df))
print("Validation size:", len(val_df))
print("Test size:", len(test_df))

train_df.iloc[3]

Train size: 8138
Validation size: 1744
Test size: 1744


Date         1979-01-03 00:00:00
EUR                        134.0
gold_diff                   -3.3
Name: 3, dtype: object

In [22]:
#give euro value => try to predict dollar value
train_df, val_df = make_date_string(train_df, val_df)

dv = DictVectorizer()

train_dict, X_train, y_train= prepare_regression_train_or_val_data(train_df, dv, True)
val_dict, X_val, y_val = prepare_regression_train_or_val_data(val_df, dv, False)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)

print(mean_squared_error(y_val, y_pred))

with open('./models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

138.93944659446228


## MLFlow part

In [ ]:
#starting an experiment
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")


2025/04/22 20:51:13 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/04/22 20:51:13 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location=('file:///d:/school/tweede_jaar/tweede '
 'semester/MLOps/MLOps-project/MLOps-project/train-and-deploy/mlruns/1'), creation_time=1745347882568, experiment_id='1', last_update_time=1745347882568, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [35]:
save_if_not_exists(train_df, val_df, test_df)

Saving files...


In [38]:
#starting an experiment run
with mlflow.start_run():

    # logging data to mlflow
    mlflow.set_tag("developer", "svb")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [41]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}



search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

d:\school\tweede_jaar\tweede semester\MLOps\MLOps-project\MLOps-project\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [21:06:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:637.64095                         
[1]	validation-rmse:471.54394                         
[2]	validation-rmse:390.84018                         
[3]	validation-rmse:355.25342                         
[4]	validation-rmse:338.80012                         
[5]	validation-rmse:333.12351                         
[6]	validation-rmse:330.28718                         
[7]	validation-rmse:328.79903                         
[8]	validation-rmse:326.65476                         
[9]	validation-rmse:326.49887                         
[10]	validation-rmse:326.35147                        
[11]	validation-rmse:326.31780                        
[12]	validation-rmse:326.31943                        
[13]	validation-rmse:326.28825                        
[14]	validation-rmse:326.27884                        
[15]	validation-rmse:326.26311                        
[16]	validation-rmse:326.28642                        
[17]	validation-rmse:326.22541                        
[18]	valid

d:\school\tweede_jaar\tweede semester\MLOps\MLOps-project\MLOps-project\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [21:06:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:930.25639                                                   
[1]	validation-rmse:897.03155                                                   
[2]	validation-rmse:865.57561                                                   
[3]	validation-rmse:835.79768                                                   
[4]	validation-rmse:807.61139                                                   
[5]	validation-rmse:780.93475                                                   
[6]	validation-rmse:756.09688                                                   
[7]	validation-rmse:732.18781                                                   
[8]	validation-rmse:709.56725                                                   
[9]	validation-rmse:688.16822                                                   
[10]	validation-rmse:667.92717                                                  
[11]	validation-rmse:648.78358                                                  
[12]	validation-rmse:630.680

...

[0]	validation-rmse:466.69349                                                    
[1]	validation-rmse:352.91763                                                    
[2]	validation-rmse:326.43870                                                    
[3]	validation-rmse:319.98338                                                    
[4]	validation-rmse:318.40280                                                    
[5]	validation-rmse:317.98122                                                    
[6]	validation-rmse:317.87546                                                    
[7]	validation-rmse:317.87650                                                    
[8]	validation-rmse:317.86726                                                    
[9]	validation-rmse:317.86641                                                    
[10]	validation-rmse:317.84631                                                   
[11]	validation-rmse:317.84669                                                   
[12]	validation-

d:\school\tweede_jaar\tweede semester\MLOps\MLOps-project\MLOps-project\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [21:19:26] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[2]	validation-rmse:329.89405                                                    
[3]	validation-rmse:321.19276                                                    
[4]	validation-rmse:318.81747                                                    
[5]	validation-rmse:318.11475                                                    
[6]	validation-rmse:318.03142                                                    
[7]	validation-rmse:317.98669                                                    
[8]	validation-rmse:318.01991                                                    
[9]	validation-rmse:318.01462                                                    
[10]	validation-rmse:318.01802                                                   
[11]	validation-rmse:318.03387                                                   
[12]	validation-rmse:318.02217                                                   
[13]	validation-rmse:317.97622                                                   
[14]	validation-

d:\school\tweede_jaar\tweede semester\MLOps\MLOps-project\MLOps-project\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [21:19:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:593.66762                                                    
[1]	validation-rmse:435.21972                                                    
[2]	validation-rmse:368.44760                                                    
[3]	validation-rmse:340.00262                                                    
[4]	validation-rmse:327.65176                                                    
[5]	validation-rmse:322.20976                                                    
[6]	validation-rmse:319.79184                                                    
[7]	validation-rmse:318.71303                                                    
[8]	validation-rmse:318.23850                                                    
[9]	validation-rmse:318.01849                                                    
[10]	validation-rmse:317.92263                                                   
[11]	validation-rmse:317.87707                                                   
[12]	validation-